$\newcommand{\pa}[1]{\left(#1\right)}$ $\newcommand{\br}[1]{\left[#1\right]}$ $\newcommand{\cbr}[1]{\left\{#1\right\}}$ $\newcommand{\abs}[1]{\left|#1\right|}$

In this file I present a problem that I ran into while running quantum circuits on IBM's quantum systems.
I do an attempt at generalizing the problem into something a bit more understandable than the quantum computing version.
I also present a couple of different techniques and which technique is the best for minimizing instances.

## 1. Problem

Two friends, Alice and Bob, just got into the business of selling apples.
The thing is, their apple orchard is half an hour, by foot, from the nearest town: the town of Quantica.
The people of Quantica love Alice and Bob's apples, and are constantly making purchases online.
Since Alice and Bob just got into the business, they lack proper equipment for apple transportation.
At the moment, their best idea is to have Alice stay at the orchard and pick the apples from the apple trees, while Bob travels by foot to Quantica to deliver the apples.
Bob has a bin of dimensions $1\times M$ apples, so he can carry at most $M$ apples per trip.

<figure>
    <img class="figure" src="../ipynb/2024-04-20/bin.png" alt="An image of Bob's bin to carry apples.">
    <figcaption><b>Figure 1.</b> A view of the bin Bob uses to carry apples. We see the first four and the last two apples, with the ellipsis indicating additional apples in between.</figcaption>
</figure>

For some reason, Alice and Bob came up with a rule for the people purchasing their apples: the amount of apples a customer buys must be exactly one more than the previous customer, with the counter reseting at the end of the day.
So, if the first customer of the day purchases $n$ apples (obviously, $n > 0$), then the next person must purchase exactly $n+1$ apples, and the next person must purchase exactly $n+2$ apples, and so on.
The people of Quantica thought this was a strange rule to implement, but continued to purchase apples nonetheless.

Of course, apples have a very short shelf life.
As a result, Alice and Bob have a limit on the maximum number of apples a person can order, call it $m$, with $n \leq m < M$.
That is, once a person places an order for $m$ apples, no additional orders can be placed for the remainder of the day.
This limit is met daily due to demand from the people of Quantica.
So, on any given day Alice and Bob receive orders for $n$, $n+1$, $n+2$, $\ldots\,$, $n + \pa{m - n -1}$, and $n + \pa{m - n}$ apples -- as $n$ progresses up to $m$.

Alice and Bob's apples are so popular that all of the orders come in almost instantly at the beginning of the day.
At first, Bob was walking the orders one by one: first walking to Quantica to take the order with $n$ apples, coming back to the apple orchard, then taking the order with $n+1$ apples, and repeating this process until the last order with $m$ apples.
This thing is, this process would result it $m-n+1$ trips to Quantica.
That's $m-n+1$ hours!

Alice and Bob thought this was way too much time and have been wanting to talk to someone about their problem.
Specifically, they want to know, 
> is there a way to plan their deliveries so that the number of trips Bob makes to Quantica is minimized?  

Knowing the problem solvers that we are, Alice and Bob have come to us with their problem.
How can we help them?

## 2. Solutions

### 2.1 Bottom-Up

I feel the first approach most people would take is to start from the smallest order, consisting of $n$ apples, and add to it orders of increasing size until $M$ apples are exceeded.
I call this technique the Bottom-Up technique.

> **Definition: Bottom-Up**  
> 
> Let $t_1\in\mathbb{Z}$, with $n < t_1 \leq m$, be s.t.
> $$\sum_{\sigma = n}^{t_1}\sigma\leq M.$$
> If $t_1 \neq m$, we seek $t_2$, $t_3$, $\ldots\,$, $t_i$, with $i\in\mathbb{Z}$, $2 \leq i \leq ?$, and $t_1 < t_2 < t_3 < \ldots < t_i = m$, s.t.
> $$\sum_{\sigma = n}^{m}\sigma = \sum_{\sigma = n}^{t_1}\sigma + \sum_{\sigma = t_1 + 1}^{t_2}\sigma + \sum_{\sigma = t_2 + 1}^{t_3}\sigma + \cdots + \sum_{\sigma = t_{i - 1} + 1}^{t_i}\sigma.$$
> Let $T := \cbr{t_1, t_2, t_3, \ldots, t_i}$.
> Then $\abs{T}$ is the number of trips taken.

That is, when $\exists\,p\in\mathbb{Z}$, with $n \leq p \leq m$, s.t.
\begin{equation} \tag{1} \sum_{i=n}^p i = \frac{p(p+1) -  n(n-1)}{2} \leq M. \end{equation}
From this relation we can see that if $p = m$ only one trip is necessary.<sup><a class="footnote" href="#fn1A">A</a></sup>
So, if 
$$M \geq \frac{m(m+1) -  n(n-1)}{2},$$
Bob must only make one trip.
Otherwise, more than one trip must be taken. 
This can be generalized with $i$ starting not necessarily at $n$.
Suppose $i$ starts at some positive integer $k$, with $n < k$, then
\begin{equation} \tag{2} \sum_{i=k}^p i = \pa{\sum_{i=0}^p i} - \pa{\sum_{i=0}^{k-1} i} = \frac{p(p+1) -  k(k-1)}{2} \leq M. \end{equation}
This process is what we code below in the function `bottom_up`.
Note that Equation (1) always works for the first $p - n$ orders, where $p$ is the value which satisfies the relation.
Then, $p + 1$ becomes $k$ in Equation (2), and we find a new value of $p$ to satisfy the new relation.
We then use this new value of $p$ to set $k = p + 1$, then repeat Equation (2) to again find a new value for $p$.
This process continues until all orders are sent; i.e., when $p = m$.

In [1]:
import numpy as np

In [2]:
def bottom_up(n, m, M):
    # Create a list of orders ranging from n to m inclusive
    orders = list(range(n, m + 1))
    trips = []  # Initialize an empty list to store the trips
    
    # Loop until all orders are processed
    while orders:
        current_trip = []  # Initialize an empty list for the current trip
        remaining_capacity = M  # Set the remaining capacity of the bin to M
        
        # Iterate over the orders in normal order (from smallest to largest)
        for order in orders:
            # If the order can fit in the remaining capacity of the bin
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
        
        # Remove the orders that were added to the current trip from the original orders list
        for order in current_trip:
            orders.remove(order)
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

In [3]:
n = 3
m = 127
M = 4092

M0 = (m * (m + 1) - n * (n - 1)) / 2

print(f"Once M >= {M0}, only one trip is required. Note that M/2 = {M0/2}.")


trips_bu = bottom_up(n, m, M)

print(f"Total trips required for top-down: {len(trips_bu)}")
print("Trips for top-down:", trips_bu)

Once M >= 8125.0, only one trip is required. Note that M/2 = 4062.5.
Total trips required for top-down: 2
Trips for top-down: [(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90), (91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127)]


In [4]:
print(np.sum(trips_bu[0]), np.sum(trips_bu[1]), len(trips_bu[0])+len(trips_bu[1]))

4092 4033 125


### 2.2 Top-Down

The main idea behind this technique is for Bob to prioritize the big orders first.
To define the Top-Down technique, consider the following summation:
$$\sum_{i = 0}^{\sigma}m-i \leq M.$$
In words, we start with the biggest order, of size $m$, and decrement order size by one until we reach an order, of size $\sigma +1$ which will push us over the threshold of $M$.
Now, this definition is not at all correct (as can be seen in my code structure) but it helps set an idea of the issues to come.
For example, suppose $n = 3$, $m = 10$ and $M = 18$.
Then, by the summation above, Bob can only take one order on the first trip, the order of size $m = 10$, since he can't take the the second order of size $m-1 = 9$ along as well.
However, by this definition we fail to even consider the possibility of taking some other order, smaller than $9$.
Bob could have taken the order of $10$ apples along with the order of $8$ apples, and he'd have left no empty slots in his bin.
So, the real definition is as follows:

> **Definition: Top-Down**  
>   
> Let $I := \cbr{i\left|i\in\mathbb{Z} \text{ and } 0\leq i\leq m-n\right.}$ and let $m_i := \cbr{m - i \left| i\in I\right.}$.
> We seek a set $T$ s.t.  
> 1. $T\subseteq m_i$,
> 2. $$\sum T \leq M,$$
> 3. $\abs{T}$ is minimal, and  
> 4. for any other set $T^\prime$ which satisfies (1), (2), and (3), $$\sum T^\prime \leq \sum T \text{ and } \text{min}\pa{T^\prime} < \text{min}\pa{T}.$$
>
> Once $T$ is found, $m_i := m_i\setminus T$ and the process is repeated.

Going back to our example of $n = 3$, $m = 10$ and $M = 18$, by this definition Bob's trips would be: $\cbr{10, 8}$ first, $\cbr{9, 7}$ second, and $\cbr{6, 5, 4, 3}$ third.
Now, we seek to find a relationship between $n$, $m$, and $M$ that will tell us how many trips Bob must take using the Top-Down technique.
We do this by considering the relationship between the value $M$ and the value of $m + n$.
Note that I continue to use the defition of $I$ throughout this section: $I := \cbr{i\left|i\in\mathbb{Z} \text{ and } 0\leq i\leq m-n\right.}$.

**Case 1: $\mathbf{M = m + n}$**  
  
Here, we can start by first sending the order with $m$ apples and the one with $n$ apples together, $T_1 = \cbr{m, n}$.
Then, on the second trip, we send the order with $m-1$ apples along with the order with $n+1$ apples, $T_2 = \cbr{m+1, n+1}$.
On the third trip, we send the order with $m-2$ apples along with the order with $n+2$ apples, $T_3 = \cbr{m-2, n+2}$.
If $m \not\equiv n\mod{2}$, we can continue this process of decrementing $m$ by 1 and incrementing $n$ by 1 until all order have been sent, making for a total of $\abs{T} = \lceil\pa{m+n}/2\rceil$ trips.<sup><a class="footnote" href="#fn2A">A</a></sup>
If $m\equiv n\mod{2}$, we see that continuing this process of decrementing $m$ by 1 and incrementing $n$ by 1 will produce a lonely order of $\pa{m+n}/2$ apples which must be sent alone.
Thus, this makes for a total of $\abs{T} = \br{\pa{m-n}/2} + 1$ trips.<sup><a class="footnote" href="#fn2B">B</a></sup>

**Case 2: $\mathbf{M < m + n}$**  
  
The largest order consists of $m$ apples.
The smallest possible order that this order can be sent with consists of $n$ apples.
However, since $M < m + n$, the order of $m$ apples must be sent by itself.
$\forall\,i\in I$, let $m_i := m - i$.
Then $m_1$ is now the order which consists of the most apples, and the smallest possible order that this order can be sent with consists of $n$ apples.
Again, if $M < m_1 +n$, the order of $m_1$ apples must be sent by itself.
We continue this process until we find some $j\in I$, with $M = m_j + n$ $\Rightarrow$ $m_j = M - n$.
This intial process takes $\abs{T} = j$ trips, where $j = m + n - M$, $\forall$ orders consisting of more than $M - n$ apples.
Once $m_j = M - n$, if $m_j \not\equiv n\mod{2}$, Bob must take an additional $\abs{T} = \lceil\pa{m+n}/2\rceil$ trips.
Else, if $m_j \equiv n\mod{2}$, Bob must take an additional $\abs{T} = \br{\pa{m-n}/2} + 1$ trips, from Case 1.

**Case 3: $\mathbf{M > m + n}$**  
  
Last is the case when $M > m + n$.
Note that we will only analyze the case when $m + n < M < m + \frac{\pa{m-n}\pa{m-n+1}}{2}$, since $\forall\, M \geq m + \frac{\pa{m-n}\pa{m-n+1}}{2}$ Bob must only take one trip.<sup><a class="footnote" href="#fn2C">C</a></sup>
Since $M > m + n$, we could implement implement an approach similar to the one we did for the case when $M = m + n$, but we omit that for now as it's code structure would not follow the same structure as the previous two cases.
Instead, we use stick with a similar approach to the previous case.
$\forall\,i\in I$, let $m_i := m - i$.
Thus, $m_0 = m$ and $m_{m - n} = n$.
Now, we know that $m_0 + m_{m - n} < M < m_0 + \frac{\pa{m_0-m_{m - n}}\pa{m_0-m_{m - n}+1}}{2}$, but we don't know where in this range $M$ lies.<sup><a class="footnote" href="#fn2D">D</a></sup>
Knowing this would yield information on the amount of orders Bob can take on the first trip.
Since $M > m_0 + m_{m - n}$, we know that at least two orders can be sent; the smallest combined quantity for two orders is $m_0$ and $m_{m-n-1}$ apples ($m_{m-n-1}$ will be found before $m_{m-n}$ in a top-down search) and the largest combined quantity for two orders being $m_0$ and $m_1$.

Now, what if $M$ is large enough to take at least three orders on the first trip?
For this to be true, $M \geq 2m + n - 1$ must be true.
In fact, this can generalized to first $p$ orders with the following function: let $\mathbb{\Lambda} := \cbr{\lambda \left| \lambda\in\mathbb{Z} \text{ and } 1\leq\lambda\leq\frac{m(m+1) -  n(n-1)}{2}\right.}$ and let $M^\ast: \Lambda\to\mathbb{Z}^+$ be defined by
$$M^\ast\pa{p} = n + \pa{p-1}\pa{\frac{2m-p+2}{2}}$$
$\forall\, p\in\Lambda$.<sup><a class="footnote" href="#fn2E">E</a></sup>
$M \geq M^\ast(p)$ must be true for Bob to be able to take at least $p$ orders on the first trip, and $M^\ast\pa{p + 1} > M \geq M^\ast(p)$ must be true for Bob to be able to take exactly $p$ orders on the first trip.
To a pretty good approximation, if the number of orders in the first trip is $\frac{3\pa{m - n}}{10}\pa{\frac{m_0(m_0+1) -  m_{m - n}(m_{m - n}-1)}{2}}$, then Bob must only take two trips to successfully deliver every order.
Thus, when 
$$M \geq n+\frac{1}{200}\pa{3m-3n-10}\pa{17m+3m+20},$$
Bob must only make two trips.<sup><a class="footnote" href="#fn2F">F</a></sup>
We can continue to make expressions like these and use them as lower bounds for $M$ to determine the amount of trips necessary, but we can do better!

Let $M_1 := m + \frac{\pa{m-n}\pa{m-n+1}}{2}$, with the subscript 1 denoting the fact that only one trip is necessary for any value of $M \geq M_1$.
Then, $M_2 := \lceil M_1/2\rceil$.
That is, $M_2$ is the value of $M$ at which two trips are necessary ($\abs{T} = 2$ if $M_2 \leq M < M_1$).
This can be generlized to $M_k := \lceil M_1/k\rceil$ for $k\in\mathbb{Z}$ and either $2 \leq k \leq\lceil\pa{m+n}/2\rceil$ for $m \not\equiv n\mod{2}$ or $2 \leq k \leq\br{\pa{m-n}/2} + 1$ for $m\equiv n\mod{2}$ (<a class="footnote" id="fnY">why?</a>).
I.e., $\abs{T} = k$ if $M_k \leq M < M_{k+1}$

In [5]:
def top_down(n, m, M):
    # Create a list of orders ranging from n to m inclusive
    orders = list(range(n, m + 1))
    trips = []  # Initialize an empty list to store the trips
    
    # Loop until all orders are processed
    while orders:
        current_trip = []  # Initialize an empty list for the current trip
        remaining_capacity = M  # Set the remaining capacity of the bin to M
        
        # Iterate over the orders in reverse (from largest to smallest)
        for order in reversed(orders):
            # If the order can fit in the remaining capacity of the bin
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
        
        # Remove the orders that were added to the current trip from the original orders list
        for order in current_trip:
            orders.remove(order)
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

### 2.3 Combined

In [6]:
def combined_approach(n, m, M):
    orders = list(range(n, m + 1))  # Create a list of orders ranging from n to m inclusive
    trips = []  # Initialize an empty list to store the trips
    
    while orders:
        largest_order = orders.pop()  # Start with the largest order
        current_trip = [largest_order]  # Initialize the current trip with the largest order
        remaining_capacity = M - largest_order  # Set the remaining capacity of the bin

        # Iterate over the orders from smallest to largest
        for order in orders[:]:
            if order <= remaining_capacity:
                current_trip.append(order)  # Add the order to the current trip
                remaining_capacity -= order  # Decrease the remaining capacity by the order size
                orders.remove(order)  # Remove the order from the original list
        
        # Add the current trip (as a tuple) to the list of trips
        trips.append(tuple(current_trip))
    
    return trips  # Return the list of trips

In [7]:
n = 14
m = 143
M = 157

trips_td = top_down(n, m, M)
trips_bu = bottom_up(n, m, M)
trips_combined = combined_approach(n, m, M)

print(f"Total trips required for top-down: {len(trips_td)}")
print("Trips for top-down:", trips_td)

print(f"Total trips required for bottom-up: {len(trips_bu)}")
print("Trips for bottom-up:", trips_bu)

print(f"Total trips required for combined approach: {len(trips_combined)}")
print("Trips for combined approach:", trips_combined)

Total trips required for top-down: 65
Trips for top-down: [(143, 14), (142, 15), (141, 16), (140, 17), (139, 18), (138, 19), (137, 20), (136, 21), (135, 22), (134, 23), (133, 24), (132, 25), (131, 26), (130, 27), (129, 28), (128, 29), (127, 30), (126, 31), (125, 32), (124, 33), (123, 34), (122, 35), (121, 36), (120, 37), (119, 38), (118, 39), (117, 40), (116, 41), (115, 42), (114, 43), (113, 44), (112, 45), (111, 46), (110, 47), (109, 48), (108, 49), (107, 50), (106, 51), (105, 52), (104, 53), (103, 54), (102, 55), (101, 56), (100, 57), (99, 58), (98, 59), (97, 60), (96, 61), (95, 62), (94, 63), (93, 64), (92, 65), (91, 66), (90, 67), (89, 68), (88, 69), (87, 70), (86, 71), (85, 72), (84, 73), (83, 74), (82, 75), (81, 76), (80, 77), (79, 78)]
Total trips required for bottom-up: 87
Trips for bottom-up: [(14, 15, 16, 17, 18, 19, 20, 21), (22, 23, 24, 25, 26, 27), (28, 29, 30, 31, 32), (33, 34, 35, 36), (37, 38, 39, 40), (41, 42, 43), (44, 45, 46), (47, 48, 49), (50, 51, 52), (53, 54), (5

## Appendix

### <p id="fn1A">1.A</p>

That is, when $\exists\,p\in\mathbb{Z}$, with $n \leq p \leq m$, s.t.
\begin{equation} \tag{1} \sum_{i=n}^p i = \frac{p(p+1) -  n(n-1)}{2} \leq M. \end{equation}
From this relation we can see that if $p = m$ only one trip is necessary.

### <p id="fn2A">2.A</p>

If $m \not\equiv m_{m-n}\mod{2}$, we can continue this process of decrementing $m$ by 1 and incrementing $n$ by 1 until all order have been sent, making for a total of $\lceil\pa{m+n}/2\rceil$ trips

### <p id="fn2B">2.B</p>

If $m\equiv n\mod{2}$, we see that continuing this process of decrementing $m$ by 1 and incrementing $n$ by 1 will produce a lonely order of $\pa{m+n}/2$ apples which must be sent alone. Thus, this makes for a total of $\br{\pa{m-n}/2} + 1$ trips.

### <p id="fn2C">2.C</p>

Note that we will only analyze the case when $m + n < M < m + \frac{\pa{m-n}\pa{m-n+1}}{2}$, since $\forall\, M \geq m + \frac{\pa{m-n}\pa{m-n+1}}{2}$ Bob must only take one trip.

### <p id="fn2D">2.D</p>

Now, we know that $m_0 + m_{m - n} < M < \frac{m_0(m_0+1) -  m_{m - n}(m_{m - n}-1)}{2}$, but we don't know where in this range $M$ lies

### <p id="fn2E">2.E</p>

In fact, this can generalized to first $p$ orders with the following function: let $\mathbb{\Lambda} := \cbr{\lambda \left| \lambda\in\mathbb{Z} \text{ and } 1\leq\lambda\leq\frac{m(m+1) -  n(n-1)}{2}\right.}$ and let $M^\ast: \Lambda\to\mathbb{Z}^+$ be defined by
$$M^\ast\pa{p} = n + \pa{p-1}\pa{\frac{2m-p+2}{2}}$$
$\forall\, p\in\Lambda$.

<figure>
    <img class="figure" src="../ipynb/2024-04-20/demo.png" alt="An image of process described above.">
    <figcaption><b>Figure 2.</b> A view of the orders Alice and Bob received. The left most column consists of $m$ apples, and the right most column of $n$ apples.</figcaption>
</figure>

*Suppose $M < 2m + n - 1$. Then $M > m_0 + m_{m-n}$ $\Rightarrow$ $\exists\, m_k\in\cbr{m_1, m_2, \ldots, m_{m-n-1}}:$ $M = m_0 + m_k$ and , with $k\in\mathbb{Z}$ and $1 \leq k \leq m - n -1$.*

### <p id="fn2F">2.F</p>

To a pretty good approximation, if the number of orders in the first trip is $\frac{3\pa{m - n}}{10}\pa{\frac{m_0(m_0+1) -  m_{m - n}(m_{m - n}-1)}{2}}$, then Bob must only take two trips to successfully deliver every order

### <p id="fn2Y">2.Y</p>

This can be generlized to $M_i := \lceil M_1/i\rceil$ for $i\in\mathbb{Z}$ and $2 \leq i \leq\pa{m+n}/2$